In [ ]:
import os
import pygmt
import pandas as pd
import xarray as xr
import numpy as np
from scripts import mapOutputs as mout

import matplotlib.pyplot as plt
%matplotlib inline

## Running the simulations

To run the different simulations, you will need to do the following in a terminal:

```bash
mpirun -np X python3 runModel.py -i input-global_plates.yml 
```

where X is the number of processors to use (for example 10).

## Visualising the outputs

You can visualise the outputs of your simulations directly in Paraview. Below we also show how this could be extracted as netcdf grids and analysed in Python.


To do so, we will be using the `mapOutputs` Python class (available in the `scripts` folder) to first extract the outputs, remap the unstructured variables on a structured mesh. The class is also performing additional calculations on the dataset to extract individual catchments based on flow accumulation and elevation.

The interpolated model's variables are then exported as `netCDF` files, and each of them contains the following (if available from the simulation):

+ elevation `elevation` (m)
+ cumulative erosion/deposition `erodep` (m)
+ erosion/deposition rate `erodep_rate` (m/yr)
+ water flux discharge (accounting for lakes) `fillDischarge`(m3/yr)
+ sediment flux in rivers `sedimentLoad` (m3/yr)
+ main basin ids `basinID` (integer)
+ rainfall `precipitation` (m/yr)
+ tectonic `uplift` (m/yr)
+ cumulative flexural isostasy `flex` (m)
  

We first define a folder where exported files will be stored:

In [ ]:
# Define output folder name for the simulation
out_path = 'export'

if not os.path.exists(out_path):
    os.makedirs(out_path)

Then we will be using the `mapOutputs` Python class to first extract the outputs, remap the unstructured variables on a structured mesh (the resolution of this mesh is defined by the `reso` parameter set in degree). The class is also performing additional calculations on the dataset to extract individual catchments based on flow accumulation and elevation.

The interpolated model's variables are then exported as `netCDF` files, and each of them contains the following:

+ elevation `elevation` (m)
+ cumulative erosion/deposition `erodep` (m)
+ water flux discharge (accounting for lakes) `fillDischarge`(m3/yr)
+ water flux in rivers (not showing lakes) `flowDischarge` (m3/yr)
+ sediment flux in rivers `sedimentLoad` (m3/yr)
+ main basin ids `basinID` (integer)
+ rainfall `precipitation` (m/yr)
+ tectonic `uplift` (m/yr)

In [ ]:
stp = 0

# Resolution of the netcdf structured grid
reso = 0.1

# Name of each netcdf output file
ncout = os.path.join(out_path, "data")

# Initialisation of the class
grid = mout.mapOutputs(path='./', filename='input-global_plates.yml', step=stp, uplift=False, flex=False)

We perform the export using the following loop:

In [ ]:
for k in range(0,11,5):
    
    if stp>1:
        # Get goSPL variables
        grid.getData(stp)
        
    # Remap the variables on the regular mesh using distance weighting interpolation
    grid.buildLonLatMesh(res=reso, nghb=3)
    
    # Export corresponding regular mesh variables as netCDF file
    grid.exportNetCDF(ncfile = ncout+str(k)+'.nc')
    stp += 1

# Plotting some outputs

In [ ]:
dataset0 = xr.open_dataset(out_path+'/data0.nc')
dataset5 = xr.open_dataset(out_path+'/data5.nc')
dataset10 = xr.open_dataset(out_path+'/data10.nc')

### Mollweide projection

In [ ]:
fig = pygmt.Figure()
# Plotting elevation
with pygmt.config(FONT='6p,Helvetica,black'):
    pygmt.makecpt(cmap="geo", series=[-10000, 10000])
    fig.basemap(region='d', projection='W6i', frame='afg')
    fig.grdimage(dataset0.elevation, shading='+a345+nt1+m0', frame=False)
    # Add contour
    fig.grdcontour(
        interval=0.1,
        grid=dataset10.elevation,
        limit=[-0.1, 0.1],
    )
    fig.colorbar(position="jBC+o0c/-1.5c+w8c/0.3c+h",frame=["a2000", "x+lElevation", "y+lm"])
# Customising the font style
fig.text(text="Step 0", position="TL", font="8p,Helvetica-Bold,black") #, xshift="-0.75c")
fig.show(dpi=500, width=1000)

fig = pygmt.Figure()
# Plotting elevation
with pygmt.config(FONT='6p,Helvetica,black'):
    pygmt.makecpt(cmap="geo", series=[-10000, 10000])
    fig.basemap(region='d', projection='W6i', frame='afg')
    fig.grdimage(dataset10.elevation, shading='+a345+nt1+m0', frame=False)
    # Add contour
    fig.grdcontour(
        interval=0.1,
        grid=dataset10.elevation,
        limit=[-0.1, 0.1],
    )
    fig.colorbar(position="jBC+o0c/-1.5c+w8c/0.3c+h",frame=["a2000", "x+lElevation", "y+lm"])
# Customising the font style
fig.text(text="Step 10", position="TL", font="8p,Helvetica-Bold,black") #, xshift="-0.75c")
fig.show(dpi=500, width=1000)

In [ ]:
fig = pygmt.Figure()
# Plotting elevation
with pygmt.config(FONT='6p,Helvetica,black'):
    pygmt.makecpt(cmap="polar", series=[-3000, 3000])
    fig.basemap(region='d', projection='W6i', frame='afg')
    fig.grdimage(dataset10.erodep, shading='+a345+nt1+m0', frame=False)
    # Add contour
    fig.grdcontour(
        interval=0.1,
        grid=dataset10.elevation,
        limit=[-0.1, 0.1],
    )
    fig.colorbar(position="jBC+o0c/-1.5c+w8c/0.3c+h",frame=["a1000", "x+lErosion/Deposition", "y+lm"])
# Customising the font style
fig.text(text="Step 10", position="TL", font="8p,Helvetica-Bold,black") #, xshift="-0.75c")
fig.show(dpi=500, width=1000)